Your company wants to build a machine learning model to predict the selling prices of houses based on a variety of features on which the value of the house is evaluated.
Your job is to build a model that will predict the price of a house based on features provided in the dataset. Senior management also wants to explore the characteristics of the houses using some business intelligence tools. One of those parameters includes understanding which factors are responsible for higher property value - $650K and above.

# Importing the necessary libraries.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import norm
from scipy import stats
from sklearn.neighbors import KNeighborsRegressor
import math
import datetime
from sklearn import linear_model 
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import PowerTransformer
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, ConfusionMatrixDisplay
from sklearn.metrics import classification_report, f1_score, cohen_kappa_score
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, OrdinalEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.metrics import cohen_kappa_score
from imblearn.over_sampling import SMOTE

In [ ]:
%matplotlib inline

In [ ]:
pd.set_option('display.max_columns',None)

# Loading the database 

In [ ]:
data = pd.read_excel('regression_data1.xls')

In [ ]:
data.head()

In [ ]:
data.shape

In [ ]:
data.columns

In [ ]:
data.dtypes

In [ ]:
data.isnull().sum()

# Analysing the independent variables

## Analysing ordinal categoricals 

In [ ]:
data['bedrooms'].value_counts()   #Perhaps removing the properties with 33 rooms. Outlier or mistake. 
#Group in less categories? 1 and 2 rooms// five or more.Imbalance in the sample. Our database is concentrated in the 3-4 rooms categories. 

In [ ]:
data['bathrooms'].value_counts() 

In [ ]:
data['floors'].value_counts() #We have mostly flats in our database and townhouses, group also here 2 floors and more. 

We have mostly flats in our database and townhouses, group also here 2 floors and more. 

In [ ]:
data['waterfront'].value_counts() 

Imbalance: most of the houses in one category. Visualizing the data with tableau, it seems that waterfront is defined very restrictively.

In [ ]:
data['view'].value_counts() 

#Most of the houses view category 0 Center of the city? lower floors in apartment buildings?

In [ ]:
data['condition'].value_counts() 

In [ ]:
#Imbalance towards average to good condition.

In [ ]:
sns.countplot('condition', data=data);
plt.show()


In [ ]:
data['grade'].value_counts() 

In [ ]:
sns.countplot('grade', data=data);
plt.show()

## Analysing date column

In [ ]:
sns.displot(data=data, x="date");


Latitute, longitude give the exact location, there is no need for the zip code. We can remove one of the two.Check which one makes more sense. Stock located in Seattle and its environments. Houses above 650k located in the waterfront, the upper part of the city, islands or proximity to water. 


## Analysing latitude and longitude

To make location meaningful for our regression, we could reference the location of each house by the proximity to locations that are valuable to inhabitants of the city, for ie distance to the airport, distance to recreational areas, to shopping centers or  distance to nature areas (parks, lakes or sea). We will use the euclidean distance to:  1)Seattle city center.The coordinates to Seattle city  are: latitude = 47.608013 and longitude = -122.335167. We calculate a new column dist_to_the_center. We try also with other coordinates such as: distance to the centre of Belle Vue (one of the neighborhoods with the most expensive houses: lat = 47.6303, lon = 122.2150; distance to the city center: lat = 47.621214 and lon = -122.349809).We will keep distance to the center of Belle Vue.

Another approach is to convert the zip code into a categorical variable and apply One Hot Encoder. 

## Analysing continuous variables


In [ ]:
sns.displot(data=data, x="sqft_living", kde=True);
plt.savefig("distribucion.png")

In [ ]:
sns.boxplot('sqft_living', data=data);
plt.savefig("Dispersion sqftliving.png")

In [ ]:
sns.displot(data=data, x="sqft_lot", kde=True); #Is this so because we have many flats? 

In [ ]:
sns.boxplot('sqft_lot', data=data);

In [ ]:
sns.displot(data=data, x="sqft_above", kde=True);

In [ ]:
sns.boxplot('sqft_above', data=data);

In [ ]:
sns.displot(data=data, x="sqft_basement", kde=True);

In [ ]:
sns.boxplot('sqft_basement', data=data);

In [ ]:
sns.displot(data=data, x="sqft_living15", kde=True);

In [ ]:
sns.boxplot("sqft_living15", data=data);

In [ ]:
sns.displot(data=data, x="sqft_lot15", kde=True);

In [ ]:
sns.boxplot("sqft_lot15", data=data);

## Analysing the dependent variable

In [ ]:
sns.displot(data=data, x="price");
plt.savefig("Distributionprice.png")

In [ ]:
sns.displot(data=data, x="price", kde=True);
plt.savefig("Distributionprice.png")

Sample collects observations between may 2014 and 2015. It seems more interesting to extract the month as an explicative variable. Seems there are four months during which sales are lower.  

Compare square living and square living 15 to see how many houses have been renovated. See if we can drop any of these columns. Plot the numeric continous variables.  

In [ ]:
sns.displot(data=data, x="price", kind="kde");

In [ ]:
sns.boxplot('price', data=data);
plt.savefig("Dispersionprice.png")

## Preparing the database for modelling

In [ ]:
df=data.copy()

#### Extracting month of the year as a new column

In [ ]:
df['month'] = df['date'].dt.month 
df.head()

#### Dropping columns and row house 33 bedrooms

In [ ]:
df.drop('date', axis=1, inplace=True)

In [ ]:
df.drop('id', axis=1, inplace=True)

In [ ]:
data[data['bedrooms']==33]   #We do not reset so we can easily locate houses df.reset_index(inplace = True)

In [ ]:
df.drop([df.index[15856]])

#### Calculating distance to the centre

In [ ]:
long_to_centre = df['long']- (-122.2150)
lat_to_centre = df['lat']-47.6303
df['dist_to_centre'] = np.sqrt(long_to_centre**2 + lat_to_centre**2)

In [ ]:
#df['lat_to_centre'] =

#### Reordering columns and adding dist_to_centre

In [ ]:
df = df[['bedrooms', 'bathrooms','floors','waterfront', 'view', 'condition', 'grade', 
          'sqft_living', 'sqft_living15', 'sqft_lot','sqft_lot15','sqft_above','sqft_basement',
          'yr_built', 'yr_renovated', 'month', 'zipcode','dist_to_centre', 'lat', 'long','price']]

In [ ]:
df.head()

## Correlations


### Correlation matrix and heatmap

In [ ]:
correlations_matrix = df.corr()
display(correlations_matrix)

In [ ]:
f,ax = plt.subplots(figsize=(16,16))
sns.heatmap(correlations_matrix, annot= True)
plt.show()

In [ ]:
#sns.pairplot(df)
#pairplotplt.show()

#### Checking for multicollinearity

We drop: sqft_above, its high correlation with sqft_living (0,88) can generate multicollinearity in the model. 

#### Selecting the variables for the model. 

In [ ]:
df1 = df[['bedrooms', 'bathrooms','floors','waterfront', 'view', 'condition', 'grade', 
          'sqft_living', 'sqft_living15', 'sqft_lot','sqft_lot15','sqft_basement',
          'yr_built', 'yr_renovated', 'dist_to_centre','price']]

# Splitting and transforming the train and test sets.

In [ ]:
y_log = np.log(df1['price'])
X = df1.drop(columns = 'price', axis =1)
X_train, X_test, y_log_train, y_log_test = train_test_split(X, y_log, test_size=0.2, random_state=200)

In [ ]:
powertransformer = PowerTransformer(method='yeo-johnson', standardize=True, copy=True)
powertransformer.fit(X_train)
X_train_trans = powertransformer.transform(X_train)
X_test_trans = powertransformer.transform(X_test)

In [ ]:
X_train_df1 = pd.DataFrame(X_train_trans, columns=X.columns)
X_test_df1 = pd.DataFrame(X_test_trans,  columns=X.columns)

In [ ]:
X_train_df1.head()

In [ ]:
X_train_df1.describe()

In [ ]:
sns.boxplot('sqft_living', data=X_train_df1);
plt.savefig("Dispersion sqftlivingafterpower.png")

# Modelling

def my_models(models, Xtrain, Xtest, ytrain, ytest):
    train_scores = []
    test_scores = []
    for model in models:
        model.fit(Xtrain, ytrain)
        ypred = model.predict(Xtrain)
        train_scores.append(r2_score(ytrain, ypred))
             
        ypred_test = model.predict(Xtest)
        test_scores.append(r2_score(ytest, ypred_test))
        
        
    scores = pd.DataFrame([train_scores, test_scores], index=["Train", "Test"], columns=models)
    
    return scores

In [ ]:
def my_models(models, Xtrain, Xtest, ytrain, ytest):
    
    metrics = {'Metrics':['MAPE','R2']*len(models), 'Model': ['KNN(k=3)','KNN(k=3)','LR','LR','MLP','MLP'],
               'Train': [], 'Test':[]}
    
    errors = {'Set': ['Train']*len(ytrain) + ['Test']*len(ytest), 'y': list(np.exp(ytrain)) + list(np.exp(ytest)),
             'y_pred_knn':[], 'y - y_pred_knn':[], 'y_pred_lr': [], 'y - y_pred_lr': [], 'y_pred_mlp': [], 
             'y - y_pred_mlp': []}
    
    for index, model in enumerate(models):
        
        model.fit(Xtrain, ytrain)
        ypred_train = model.predict(Xtrain)
        ypred_test = model.predict(Xtest)
        
        if ( index == 0 ):
            errors['y_pred_knn'].extend(list(np.exp(ypred_train)))
            errors['y_pred_knn'].extend(list(np.exp(ypred_test)))
            errors['y - y_pred_knn'].extend(list(np.exp(ytrain)-np.exp(ypred_train)))
            errors['y - y_pred_knn'].extend(list(np.exp(ytest)-np.exp(ypred_test)))        
        elif ( index == 1):
            errors['y_pred_lr'].extend(list(np.exp(ypred_train)))
            errors['y_pred_lr'].extend(list(np.exp(ypred_test)))
            errors['y - y_pred_lr'].extend(list(np.exp(ytrain)-np.exp(ypred_train)))
            errors['y - y_pred_lr'].extend(list(np.exp(ytest)-np.exp(ypred_test)))
        else:
            errors['y_pred_mlp'].extend(list(np.exp(ypred_train)))
            errors['y_pred_mlp'].extend(list(np.exp(ypred_test)))
            errors['y - y_pred_mlp'].extend(list(np.exp(ytrain)-np.exp(ypred_train)))
            errors['y - y_pred_mlp'].extend(list(np.exp(ytest)-np.exp(ypred_test)))
  
        metrics['Train'].append(mean_absolute_percentage_error(np.exp(ytrain),np.exp(ypred_train)))
        metrics['Train'].append(r2_score(np.exp(ytrain),np.exp(ypred_train)))
        metrics['Test'].append(mean_absolute_percentage_error(np.exp(ytest),np.exp(ypred_test)))
        metrics['Test'].append(r2_score(np.exp(ytest),np.exp(ypred_test)))
                
    metrics_df = pd.DataFrame(metrics)
    errors_df = pd.DataFrame(errors)                                      
    return metrics_df, errors_df

We calculate the r2scores and the MAPE for the three different models we used.

In [ ]:
models = [KNeighborsRegressor(n_neighbors=3), linear_model.LinearRegression(), MLPRegressor()]

scores, results = my_models(models, X_train_trans, X_test_trans, y_log_train, y_log_test)
display(scores)
display(results)

In [ ]:
fig, axes = plt.subplots(3,3, figsize=(16,10), dpi=300)

axes[0,0].scatter(x=results.y,y=results.y_pred_knn);
axes[0,0].plot(results.y,results.y,color="black");
axes[0,1].scatter(y=results['y - y_pred_knn'],x=results.y);
axes[0,2].hist(results['y - y_pred_knn'], bins= 50);

axes[0,0].set_xlabel("Real price")
axes[0,0].set_ylabel("Predicted price")
axes[0,0].set_title("K-nn(k=3)")

axes[0,1].set_ylabel("error")
axes[0,1].set_xlabel("Real price")
axes[0,1].set_title("K-nn(k=3)")

axes[0,2].set_title("K-nn error distribution")
axes[0,2].set_xlabel("Error")

axes[1,0].scatter(x=results.y,y=results.y_pred_lr);
axes[1,0].plot(results.y,results.y,color="black");
axes[1,1].scatter(y=results['y - y_pred_lr'], x=results.y);
axes[1,2].hist(results['y - y_pred_lr'], bins=50);


axes[1,0].set_xlabel("Real price")
axes[1,0].set_ylabel("Predicted price")
axes[1,0].set_title("linear model")

axes[1,1].set_ylabel("error")
axes[1,1].set_xlabel("Real price")
axes[1,1].set_title("linear model")

axes[1,2].set_title("linear model error distribution")
axes[1,2].set_xlabel("Error")

axes[2,0].scatter(x=results.y,y=results.y_pred_mlp);
axes[2,0].plot(results.y,results.y,color="black");
axes[2,1].scatter(y=results['y - y_pred_mlp'], x=results.y);
axes[2,2].hist(results['y - y_pred_mlp'],bins=50);

axes[2,0].set_xlabel("Real price")
axes[2,0].set_ylabel("Predicted price")
axes[2,0].set_title("MLP")

axes[2,1].set_ylabel("error")
axes[2,1].set_xlabel("Real price")
axes[2,1].set_title("MLP")

axes[2,2].set_title("MLP error distribution")
axes[2,2].set_xlabel("Error")




plt.tight_layout()
plt.savefig("resultados_regression.png")
plt.show()

MLP best prediction performance, errors increase as prices increase. Our model predicts worse these transactions. Next steps to increase the predictive capacity of the models: remove outliers (in price and size variables), dummify zip codes.